In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/llm-detect-ai-generated-text/sample_submission.csv
/kaggle/input/llm-detect-ai-generated-text/train_prompts.csv
/kaggle/input/llm-detect-ai-generated-text/test_essays.csv
/kaggle/input/llm-detect-ai-generated-text/train_essays.csv


In [4]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

In [5]:
# Load the training data
train_essays = pd.read_csv("/kaggle/input/llm-detect-ai-generated-text/train_essays.csv")
train_prompts = pd.read_csv("/kaggle/input/llm-detect-ai-generated-text/train_prompts.csv")




In [6]:
# Merge training essays with prompts based on prompt_id
train_data = pd.merge(train_essays, train_prompts, on="prompt_id", how="left")

In [7]:
# Data preprocessing
# Assuming 'text' is the column containing essay text, 'generated' is the target field
X = train_data['text']
y = train_data['generated']

In [8]:
# Feature extraction using TF-IDF
tfidf_vectorizer = TfidfVectorizer(max_features=5000)
X_tfidf = tfidf_vectorizer.fit_transform(X)

In [9]:
# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X_tfidf, y, test_size=0.2, random_state=42)

In [10]:
# Train an SVM classifier
# Using a linear kernel and a pipeline with standard scaling for better performance
svm_model = make_pipeline(StandardScaler(with_mean=False), SVC(kernel='linear', C=1.0, random_state=42))
svm_model.fit(X_train, y_train)

Pipeline(steps=[('standardscaler', StandardScaler(with_mean=False)),
                ('svc', SVC(kernel='linear', random_state=42))])

In [11]:
# Evaluate the SVM model
predictions = svm_model.predict(X_val)
accuracy = accuracy_score(y_val, predictions)
print(f"Accuracy: {accuracy}")
print(classification_report(y_val, predictions))

Accuracy: 0.9963768115942029
              precision    recall  f1-score   support

           0       1.00      1.00      1.00       275
           1       0.00      0.00      0.00         1

    accuracy                           1.00       276
   macro avg       0.50      0.50      0.50       276
weighted avg       0.99      1.00      0.99       276



/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [12]:
# Load the test data
test_data = pd.read_csv("/kaggle/input/llm-detect-ai-generated-text/test_essays.csv")

In [13]:
# Assuming 'text' is the column containing essay text
X_test = test_data['text']

In [14]:
# Feature extraction using the same TF-IDF vectorizer used during training
X_test_tfidf = tfidf_vectorizer.transform(X_test)

In [15]:
# Make predictions using the trained SVM model
test_predictions = svm_model.predict(X_test_tfidf)

In [16]:
# Create a DataFrame for the submission
submission_df = pd.DataFrame({
    'id': test_data['id'],
    'generated': test_predictions
})

In [17]:
# Save the submission DataFrame to a CSV file
submission_df.to_csv("submission.csv", index=False)

# Display the first few rows of the submission file
print(submission_df.head())

         id  generated
0  0000aaaa          0
1  1111bbbb          0
2  2222cccc          0
